# 모델링
- 레이블 인코딩한 데이터
    * 오버샘플링 하지 않은 데이터 - 학습시 Cost-sensitive learning 사용해 클래스 불균형 해소 예정
    * SMOTE NC 오버샘플링한 데이터
<br><br>
- 원-핫 인코딩한 데이터-> ``메모리 과부하 걸려서 Kernel이 죽어버림..``
    * 오버샘플링 하지 않은 데이터 - 학습시 Cost-sensitive learning 사용해 클래스 불균형 해소 예정
    * SMOTE NC 오버샘플링한 데이터 
    
<br>

- 선형기반 알고리즘인 Logistic Regression 하나 적용
- 트리기반 알고리즘인 Decision Tree 하나 적용
- 두 모델에서 각각 가장 성능이 좋은 데이터 종류 하나씩 선정
- Train : Test 비율은 7 : 3으로 적용

In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('/Users/younghun/Desktop/gitrepo/data/healthcare/encoding_df')

In [2]:
label_df = pd.read_csv('label_df.csv')
#label_smote_df = pd.read_csv('label_smote_df.csv')
#ohe_df = pd.read_csv('ohe_df.csv')
#ohe_smote_df = pd.read_csv('ohe_smote_df.csv')

## Multi-Logistic Regression

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

feature = label_df.iloc[:, :-1]
target = label_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, random_state=42)

lr_clf = LogisticRegression(multi_class='multinomial')
lr_clf.fit(X_train, y_train)
y_pred = lr_clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test 데이터에 대한 정확도: {acc: .4f}")

Test 데이터에 대한 정확도:  0.2808


/Users/younghun/opt/anaconda3/envs/venvforpython/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## Random Forest 
- 레이블 인코딩은 트리 기반에 강하다..!

In [5]:
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, random_state=42)

rf_clf = RandomForestClassifier(n_estimators=200,
                               criterion='entropy',
                               min_samples_split=10)
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"테스트 데이터에 대한 정확도: {acc: .4f}")

테스트 데이터에 대한 정확도:  0.3878


## Light GBM
- Light GBM도 결과가 좋지 않다면... Tensorflow 딥러닝을 활용해보자

In [10]:
label_df.head(1)

,Unnamed0,Hospitaltypecode,CityCodeHospital,Hospitalregioncode,Department,WardType,WardFacilityCode,TypeofAdmission,SeverityofIllness,Age,Hospitaltypeabinary,AvailableExtraRoomsinHospital,VisitorswithPatient,AdmissionDeposit,VisitorsRobust,VisitorsQ3ohe,AdmissionDepositRobust,AdmissionDepositQ3ohe,AdmissionDepositDiscretization,YStay
0,0,2,2,2,3,2,5,0,0,5,0.0,3,2,4911.0,-0.5,0.0,0.139002,0.0,5.0,0


In [9]:
import re
label_df = label_df.rename(columns= lambda x : re.sub('[^A-Za-z0-9]+', '', x))

In [13]:
from lightgbm import LGBMClassifier

feature = label_df.iloc[:, :-1]
target = label_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(feature,
                                                   target,
                                                   test_size=0.3,
                                                   random_state=12)
lgbm = LGBMClassifier(objective='multiclass',
                      n_estimators=400, learning_rate=0.1,
                     random_state=42)
lgbm.fit(X_train, y_train)
y_pred = lgbm.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"테스트 데이터에 대한 정확도: {acc: .4f}")

테스트 데이터에 대한 정확도:  0.4032


# Tensorflow 이용해야 할듯...